In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing titanic data from the below link:
https://www.kaggle.com/azeembootwala/titanic

In [ ]:
input_ads_pre = pd.read_csv('../input/titanic/train_data.csv')
input_ads_pre.drop(columns=['Unnamed: 0','Title_1','Title_2','Title_3','Title_4'],inplace=True) #Dropping un-necessary columns
#-----------------------------------------------------------------
print(input_ads_pre.shape)
input_ads_pre.head()

# Null Check

In [ ]:
pd.DataFrame(input_ads_pre.isnull().sum()).T

# Describing the dataset

In [ ]:
input_ads_pre.describe()

# Describing the target variable

In [ ]:
#Total survived vs not-survived split in the training data
input_ads_pre['Survived'].value_counts()

# Shuffling the data

In [ ]:
from sklearn.utils import shuffle
#np.random.seed(100)

#----------------------------------------------------
input_ads = shuffle(input_ads_pre,random_state=100)
print(input_ads.shape)
input_ads = input_ads.reset_index(drop=True)
input_ads.head(3)

# Data Manipulation of Train and Test

In [ ]:
target = 'Survived' #To predict

#--------------------------------------------------------------------------------
#Splitting into X & Y datasets (supervised training)
X = input_ads[[cols for cols in list(input_ads.columns) if target not in cols]]
y = input_ads[target]

print(X.columns)

#--------------------------------------------------------------------------------
#Since test data is already placed in the input folder separately, we will just import it
test_ads_pre = pd.read_csv('../input/titanic/test_data.csv')
test_ads_pre.drop(columns=['Unnamed: 0','Title_1','Title_2','Title_3','Title_4'],inplace=True) #Dropping un-necessary columns
test_ads = shuffle(test_ads_pre,random_state=100)
test_ads = test_ads.reset_index(drop=True)

#Splitting into X & Y datasets (supervised training)
X_test = test_ads[[cols for cols in list(test_ads.columns) if target not in cols]]
y_test = test_ads[target]

print('Train % of total data:',100 * X.shape[0]/(X.shape[0] + X_test.shape[0]))
#--------------------------------------------------------------------------------
#Manipulation of datasets for convenience and consistency
X_arr = np.array(X)
X_test_arr = np.array(X_test)

y_arr = np.array(y).reshape(X_arr.shape[0],1)
y_test_arr = np.array(y_test).reshape(X_test_arr.shape[0],1)

#--------------------------------------------------------------------------------
#Basic Summary
print(X_arr.shape)
print(X_test_arr.shape)
print(y_arr.shape)

# Decision Tree Classifier from scratch

## UDF to calculate gini index of each node

In [ ]:
def gini_node(arr_,k):

    class_elem_total = 0
    for class_ in k: #Iterating for each class in the node
        class_elem = (np.sum((arr_==class_).astype(int)))/len(arr_)
        class_elem = class_elem**2
        class_elem_total = class_elem_total + class_elem 
        
    gini_node = 1 - class_elem_total
    return gini_node

## UDF for Gini index of a split (Weighted by the leafs)

In [ ]:
def gini_split(left_arr,right_arr,k):

    #Total obs in each node
    m_left = len(left_arr)
    m_right = len(right_arr)
    m_total_node = m_left + m_right  

    #Calculating gini index for each 
    gini_left = gini_node(left_arr,k)
    #print(gini_left)
    gini_right = gini_node(right_arr,k)
    #print(gini_right)
    
    #Calculation of gini for the split
    if m_left==0:
        gini_split = ((m_right/m_total_node) * gini_right)
        
    elif m_right==0:
        gini_split = ((m_left/m_total_node) * gini_left)
    
    elif (m_left>0) & (m_right>0):
        gini_split = ((m_left/m_total_node) * gini_left) + ((m_right/m_total_node) * gini_right)

    return gini_split

## UDF for inding best split for a feature (Greedy Exact Search)

In [ ]:
def feature_split_algo(data,col_idx,min_samples_split=2,min_samples_leaf=2):
    
    assert len(data[:,col_idx])>=min_samples_split, "Data Insufficient - Include more data or reduce min_samples_split hyper-param"
    
    k = np.unique(data[:,-1]) #Unique of all classes going into to function
    
    #------------------------------------------------------------------------------------------------------
    #To be used for thresholds
    unique_vals = np.unique(data[:,col_idx])
    
    #print('Total unique vals in the column :',len(unique_vals))
    
    if len(unique_vals)>1:

        gini_node_ = gini_node(data[:,-1],k)

        splits_gini_dict = {}
        thresholds_discarded = []

        #------------------------------------------------------------------------------------------------------
        for threshold in unique_vals: #For each threshold possible

            #print('-------- Threshold : -------',threshold)

            left_split = data[data[:,col_idx]<=threshold] #Left extension of tree
            left_split_target = left_split[:,-1]
            #print('Len left :',len(left_split_target))

            right_split = data[data[:,col_idx]>threshold] #Right extension of tree
            right_split_target = right_split[:,-1]
            #print('Len right :',len(right_split_target))

            #--------------------------------------------------------------------------------------------------
            if (len(left_split_target)>=min_samples_leaf) & (len(right_split_target)>=min_samples_leaf): #Condition on mininum samples for a split to be eligible
                
                #Calculating the gini index of the split
                gini_split_ = gini_split(left_arr=left_split_target,
                                         right_arr=right_split_target,
                                         k=k)
                #print(gini_split_)
                    
                splits_gini_dict.update({threshold : gini_split_})

            else:
                #Discarding the threshold if condition not met
                thresholds_discarded.append(threshold)

        #print('Thresholds discarded :',thresholds_discarded)
        #print(splits_gini_list)
        #------------------------------------------------------------------------------------------------------
        # Finding min value keys in dictionary

        #assert len(splits_gini_dict)>0, "No thresholds evaluated"
        
        
        #Condition to avoid empty dictionary (if no split is feasible)
        if len(splits_gini_dict)>0:

            min_gini = min(splits_gini_dict.values())
            split_val = [key for key in splits_gini_dict if splits_gini_dict[key] == min_gini]
            #print('Min Score :',min_gini)
            #print('Min Score Split Value :',split_val[0])
            #------------------------------------------------------------------------------------------------------
            split_col_map_dict = {col_idx : split_val[0]}
            best_score_col_map_dict = {col_idx : min_gini}

            return split_col_map_dict,best_score_col_map_dict
        
        else:
            #print('1. No Split')
            split_col_map_dict = {col_idx : np.nan}
            best_score_col_map_dict = {col_idx : np.nan}
        
            return split_col_map_dict,best_score_col_map_dict
            
    
    else:
        #print('2. No Split')
        split_col_map_dict = {col_idx : np.nan}
        best_score_col_map_dict = {col_idx : np.nan}
        
        #Returning dict of the best split and their best score with their col idx as key
        return split_col_map_dict,best_score_col_map_dict


## UDF for overall best split finding algorithm (Greedy Exact Search)

In [ ]:
def overall_split(data,col_idx_eligible):
    
    scores_dict = {}
    split_val_dict = {}
    #col_idx_eligible = list(range(data.shape[1])) 

    #--------------------------------------------------------------------------------------------------
    #For a subset of columns required
    for col_idx in col_idx_eligible:

        #print('\n#--------- Index of columns :',col_idx,' ---------#\n')
        #Finding the best split for the feature
        split_val_dict_temp,scores_dict_temp = feature_split_algo(data=data,
                                                                   col_idx=col_idx)
        print(scores_dict_temp)

        scores_dict.update(scores_dict_temp)
        split_val_dict.update(split_val_dict_temp)

    #-------------------------------------------------------------------------------------------------
    best_score_overall = min(scores_dict.values()) #Extracting the min score across all columns
    best_score_col_idx = [key for key in list(scores_dict.keys()) if scores_dict[key]==best_score_overall] #Extracting the col idx with the best score
    #print('best_score_col_idx:',best_score_col_idx)
    best_col_idx_split_val = split_val_dict[best_score_col_idx[0]]
    
    #Returning dict of col idx and lsit of best split score and corresponding threshold
    return {best_score_col_idx[0] : [best_score_overall,best_col_idx_split_val]}


## UDF for splitting the current dataframe by the best split found out through the above algorithm

In [ ]:
def split_array(arr,arr_y,col_idx,split_val):
    
    #print('Col idx :',col_idx,'-------> Split value :',split_val)
    
    #Split for X of data
    x_left = arr[arr[:,col_idx]<=split_val]
    x_right = arr[arr[:,col_idx]>split_val]
    
    #Split for Y of data
    y_left = arr_y[arr[:,col_idx]<=split_val]
    y_right = arr_y[arr[:,col_idx]>split_val]
    
    return x_left,x_right,col_idx,split_val,y_left,y_right

## Class for the Classification Tree

In [ ]:
class cart:
    
    def __init__(self):
            print("in init")
    
    #UDF for CART
    def grow_tree(self,x_data,y_data,node_dict,max_depth,feat_idx_list=[0,1,4,5,7,9],min_samples_split=5,min_samples_leaf=2,depth=0,input_ads_=input_ads):
        
        input_ads_ = input_ads_.drop(columns=target) #Dropping target column
        
        #To restrict going above max depth
        if depth<=max_depth:
            
            print('#---------------------------------- DEPTH :',depth,' -----------------------------------#')
            
            #Calculating best split overall
            split_dict = overall_split(data=np.append(x_data,y_data,axis=-1),
                                       col_idx_eligible=feat_idx_list)

            split_col_idx = list(split_dict.keys())[0]
            gini_ = list(split_dict.values())[0][0]
            split_col_val = list(split_dict.values())[0][1]
            
            #--------------------------------------------------------------------------------------------------------------
            print('1. -------> Entering root_node of depth :',depth)
            #Splitting on the best split point found
            x_left,x_right,col_idx,split_val,y_left,y_right = split_array(arr=x_data,arr_y=y_data,
                                                                          col_idx=split_col_idx,
                                                                          split_val=split_col_val)
            
            #Defining dictionary for the node of tree 
            node_dict = {'col': input_ads_.columns[split_col_idx], 'col_idx':split_col_idx,
                         'threshold':split_col_val,'val': np.mean(y_data),'n_class_0': len(y_data[y_data==0]),
                         'n_class_1': len(y_data[y_data==1]),'n_vals':len(y_data)}  # save the information 
            
            #-----------------------------------------------------------------------
            print('2. ------->First :\n',node_dict)

            # generate tree for the left hand side data
            print('3. -------> Entering left of depth:',depth)
            node_dict['left'] = self.grow_tree(x_data=x_left,
                                               y_data=y_left,
                                               feat_idx_list=feat_idx_list,
                                               node_dict={},
                                               max_depth=max_depth,
                                               min_samples_split=min_samples_split,
                                               min_samples_leaf=min_samples_leaf,
                                               depth=depth+1)   
            #-----------------------------------------------------------------------
            if node_dict['left']==None:
                print('4. -------> None:\n')
            
            # right hand side trees
            print('5. -------> Entering right of depth:',depth)
            node_dict['right'] = self.grow_tree(x_data=x_right,
                                               y_data=y_right,
                                               feat_idx_list=feat_idx_list,
                                               node_dict={},
                                               max_depth=max_depth,
                                               min_samples_split=min_samples_split,
                                               min_samples_leaf=min_samples_leaf,
                                               depth=depth+1)
            if node_dict['right']==None:
                print('6. --------> None:\n')
            
            #print('After :\n',node_dict)
            #Error Handling
            try:
                self.depth += 1   # increase the depth since we call fit once
            except:
                print('7. -------> Entering except---')
                return node_dict
            
        elif depth>max_depth:
            return None
        
        elif (len(y_data)<min_samples_split) | (len(y_data)<min_samples_leaf):
            return None
        
        elif node_dict is None:
            return None

        
        #Returns the fully expanded tree
        return node_dict
    
    

## Invoking the UDF for the whole classification tree building 

In [ ]:
cart_ = cart()
tree_dict_ = cart_.grow_tree(x_data=X_arr,
                             y_data=y_arr,
                             node_dict={},
                             max_depth=2,
                             input_ads_=input_ads)

## UDF for prediction of a single row in the test data

In [ ]:
def single_row_pred(test_x_,max_depth,temp_tree_dict): #Takes in the tree dict from training

    for i in range(max_depth): #For all depth
        
        #print('------ depth :',i)
        threshold = temp_tree_dict['threshold']
        split_col_idx = temp_tree_dict['col_idx']

        tree_dict_left = temp_tree_dict['left']
        tree_dict_right = temp_tree_dict['right']
        
        #Traversing into left side
        if (test_x_[split_col_idx]<=threshold) & (tree_dict_left!=None) & (tree_dict_right!=None):

            temp_tree_dict = tree_dict_left

            if (temp_tree_dict['left']==None) & (temp_tree_dict['right']==None):
                prediction = temp_tree_dict['val']
                #pred_list.append(prediction)

        #Traversing into right side
        elif (test_x_[split_col_idx]>threshold) & (tree_dict_left!=None) & (tree_dict_right!=None):

            temp_tree_dict = tree_dict_right

            if (temp_tree_dict['left']==None) & (temp_tree_dict['right']==None):
                prediction = temp_tree_dict['val']
                #pred_list.append(prediction)

        #If end of the tree is reached, generate predictions 
        elif (tree_dict_left==None) & (tree_dict_right==None):

            prediction = temp_tree_dict['val']
            
            
    return prediction

## UDF for prediction overall 

In [ ]:
def predict_tree(test_data,max_depth,tree_object=tree_dict_,threshold=0.5): #Takes in tree dictionary and threshold of proabability
    
    pred_list = []
    
    #For each row in test data
    for idx in range(len(test_data)):
        
        #Sngle row prediction calculation
        pred = np.round(single_row_pred(test_x_=test_data[idx],
                                        max_depth=max_depth,
                                        temp_tree_dict=tree_object),3)
        pred_list.append(pred)
        
    print('Length of preds :',len(pred_list))
    
    #Converting into array
    pred_proba = np.array(pred_list)
    
    #Converting into class predictions (binary) based on threshold
    pred_list = (np.array(pred_proba)>threshold).astype(int)
        
    return pred_list

## Invoking prediction UDF (Generating predictions from the manual decision tree classification model on the test data)

In [ ]:
preds_manual = predict_tree(test_data=X_test_arr
                            ,max_depth=2,
                            tree_object=tree_dict_,
                            threshold=0.5)

#----------------------------------------------------------------------------------------------------
print('Total predictions :', len(preds_manual))
print('Unique of predictions :',np.unique(preds_manual))
preds_manual[0:10]

#----------------------------------------------------------------------------------------------------
#Evaluating the model
score = roc_auc_score(y_test_arr, preds_manual)
print('1. ROC AUC: %.3f' % score)
print('2. Accuracy :',accuracy_score(y_test_arr, preds_manual))
print('3. Classification Report -\n',classification_report(y_test_arr, preds_manual))
print('4. Confusion Matrix - \n',confusion_matrix(y_test_arr, preds_manual))

## Insights : We got an accuracy of 81% with ~0.78 ROC AUC, very good for a manually implemented model!!!

# Sklearn Benchmarking

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=100,max_depth=2,min_samples_split=5,min_samples_leaf=2)
dt_clf.fit(X_arr[:,[0,1,4,5,7,9]],y_arr)

sklearn_preds = dt_clf.predict(X_test_arr[:,[0,1,4,5,7,9]])

#------------------------------------------------------------------------------------------------------
#Evaluating the model
score = roc_auc_score(y_test_arr, sklearn_preds)
print('1. ROC AUC: %.3f' % score)
print('2. Accuracy :',accuracy_score(y_test_arr, sklearn_preds))
print('3. Classification Report -\n',classification_report(y_test_arr, sklearn_preds))
print('4. Confusion Matrix - \n',confusion_matrix(y_test_arr, sklearn_preds))

## Insights : The manual implementation performance is exactly the same as the skelarn version with same hyper-params, indicating correct implementation!! 

# END